In [5]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os
import glob

from scipy.signal import find_peaks
from sklearn.preprocessing import StandardScaler
from skimage.restoration import denoise_tv_chambolle

from tqdm import tqdm_notebook
from IPython import display

**Summary**

This notebook generate matrix with features for matricies with Fourier spectrum for electrode and optical mapping. 

Features description:

-  *freq i* - frequency of i$^{th}$ heightest peak
-  *height i* - height of i$^{th}$ heightest peak
-  *width i* - width of i$^{th}$ heightest peak
-  *prominence i* - prominence of i$^{th}$ heightest peak
-  *#peaks_th* - number of peaks for given (th) threshhold
-  *low_freq_noise* - presence of low-frequency noise (frequency of one of the n highest peaks in the interval from 0 to lf_thHz)


### Upload data

In [7]:
path = r'C:\Users\ecath\Desktop\Research\Raw Data'

mem_spectrum = pd.read_csv(path + '\LD dataset spectrum\Spectrum of electrode LD.csv', index_col=0)
niom_spectrum = pd.read_csv(path + '\LD dataset spectrum\Spectrum of optical LD.csv', index_col=0)

C:\Users\ecath\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Scaling

In [3]:
def scaling(df):  
    scaler = StandardScaler() 

    y_col = [col for col in df.columns if '_yf' in col] 

    df_yf = df[y_col]
    target = pd.DataFrame(df_yf.transpose().target).transpose()
    df_yf = df_yf.drop(['target'])
    df_yf = df_yf.replace(0, np.nan)

    scaled_features = scaler.fit_transform(df_yf.values)
    df_ = pd.DataFrame(scaled_features, columns=df_yf.columns, index=df_yf.index)
    df_ = df_.fillna(value=0, axis=1)
    df_ = pd.concat([df_, target], axis = 0)   
    df[y_col] = df_
    return(df)

In [8]:
mem_spec = scaling(mem_spectrum)
niom_spec = scaling(niom_spectrum)

C:\Users\ecath\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:776: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\ecath\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:781: RuntimeWarning: Degrees of freedom <= 0 for slice.
  new_unnormalized_variance = np.nanvar(X, axis=0) * new_sample_count
C:\Users\ecath\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:776: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
C:\Users\ecath\Anaconda3\lib\site-packages\sklearn\utils\extmath.py:781: RuntimeWarning: Degrees of freedom <= 0 for slice.
  new_unnormalized_variance = np.nanvar(X, axis=0) * new_sample_count


### Feature generation

In [9]:
"""
Function that generates pd.DataFrame with amount of peaks for (th*100)% threshhold. 

Parameters: 

full_df: Dataframe
Dataframe with fourier spectrum

th: float, from 0 to 1
threshhold

Returns: 

features: DataFrame, shape=(full_df[1]/2, 1)
number of peaks
"""

def number_of_peaks(full_df, th):
    all_props = []
    df = full_df[full_df.columns[::2]][:-1]
    for col in df:
        _, properties = find_peaks(df[col][df[col] != 0], height=0)
        all_props.append(properties)
    num_of_peaks = []
    
    for i in range(len(all_props)):
        try:
            max_height = np.max(all_props[i]['peak_heights'])
            peaks, _ = find_peaks(df.iloc[:,i], threshold=th)
            num = peaks.shape[0]
        except ValueError:
            num = 0
        num_of_peaks.append(num)
    num_of_peaks = pd.DataFrame(num_of_peaks, columns=['#peaks_' + str(th)])
    return(num_of_peaks)

In [10]:
number_of_peaks(mem_spec, 2.5).describe()

,#peaks_2.5
count,1728.000000
mean,1.547454
std,0.925765
min,0.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,6.000000


In [12]:
def get_props(full_df):
    all_peaks = []
    all_props = []

    df = full_df[full_df.columns[::2]][:-1]
    xf = full_df[full_df.columns[1::2]][:-1]

    for col in df:
        peaks, properties = find_peaks(df[col][df[col] != 0], height=0, width=0, prominence=0, rel_height=0.5)
        all_props.append(properties)
        all_peaks.append(peaks)
    return df, xf, all_peaks, all_props
 

        
def height_width_prominence(full_df, n, prop):
    
    df, xf, all_peaks, all_props = get_props(full_df)
    all_max_prop = []

    
    for i in range(len(all_props)):
        try:
            z = np.argsort(all_props[i]['peak_heights'])
            z = z[:-(n+1):-1]
            
            n_max_prop = all_props[i][prop][z] #heights of max peaks
            all_max_prop.append(n_max_prop)
            
        except IndexError:
            n_max_prop = np.zeros((n))
            all_max_prop.append(n_max_prop)
            
    all_max_prop = pd.DataFrame(all_max_prop, columns=[prop + ' ' + str(i) for i in range(n)])      
    return all_max_prop


def freq_and_label(full_df, n):
    
    df, xf, all_peaks, all_props = get_props(full_df)

    freq = []
    el_om_labels = []

    
    for i in range(len(all_props)):
        try:
            z = np.argsort(all_props[i]['peak_heights'])
            z = z[:-(n+1):-1]
            
            fr = xf.iloc[:,i][all_peaks[i][z]].values #freqs of max peaks
            freq.append(fr)
            
            el_om_label = 1
            el_om_labels.append(el_om_label)
            
        except IndexError:
            fr = np.zeros((n))
            freq.append(fr)
            
            el_om_label = 0
            el_om_labels.append(el_om_label)
    
    freq = pd.DataFrame(freq, columns=['freq ' + str(i) for i in range(n)])   
    el_om_labels = pd.DataFrame(el_om_labels, columns=['Label for OM and EL'])
    return freq, el_om_labels

def second_harm(full_df, n): 
    
    df, xf, all_peaks, all_props = get_props(full_df)

    
    freq = []
    given_n = n
    
    for i in range(len(all_props)):
        try:
            z = np.argsort(all_props[i]['peak_heights'])
            z = z[:-(n+1):-1]
            
            if len(all_props[i]['peak_heights']) < given_n:
                range_n = len(all_props[i]['peak_heights'])
            else: 
                range_n = given_n
            
            fr_forharm = xf.iloc[:,i][all_peaks[i][z]] # list of frequencies for max peaks
            fr_forharm.reset_index(drop = True, inplace = True)

            index_forharm = 0 #initiation of second harmonics index (for each i) 
            for q in range(range_n):
                for p in range(range_n):
                    try:
                        a = fr_forharm[p] / fr_forharm[q] #frequencies relation
                        if (a < 2.1) and (a > 1.9): 
                            index_forharm = 1 # if relation is 2 plus/minus 5% output 1 
                    except ZeroDivisionError:
                        a = 0 
            freq.append(index_forharm)
            
        except IndexError:
            index_forharm = 0
            freq.append(index_forharm)
    
    freq = pd.DataFrame(freq, columns=['second_harmonics'])

    return freq


def concat(full_df, n):
    
    height = height_width_prominence(full_df, n, 'peak_heights')
    width = height_width_prominence(full_df, n, 'widths')
    prominence = height_width_prominence(full_df, n, 'prominences')
    freq, label = freq_and_label(full_df, n)
    freq_forharm = second_harm(full_df, n) 
    
    features = pd.concat([freq, height, width, prominence, freq_forharm, label], axis=1)
    features.fillna(0.0, inplace=True)
    return(features)

"""
Calculate the SNR value for MEM and NIOM spectrum

Parameters: 
-----------
df: pd.DataFrame
    n x m dataframe with spectrums 

Returns: 
-------
snr: pd.DataFrame
    1 x n dataframe with correspondings SNR values
    
"""

def SNR(df):
    y_col = [col for col in df.columns if '_yf' in col]
    y_df = df[y_col]
    snr = []
        
    for i in range(y_df.shape[1]):          
        s = y_df[y_col[i]][y_df[y_col[i]] != 0]
        _, properties = find_peaks(s, height=0)
        
        num_of_avg_peak = 2 #number of highest peak tp average
        mean_max = np.mean(np.sort(properties['peak_heights'])[-num_of_avg_peak:]) 
        
        sd = s.std(axis=0)
        ratio = np.round(mean_max / sd, 2)
        ratio = np.where(sd == 0, 0, ratio)
        snr.append(ratio)
        
    snr = pd.DataFrame(snr, columns=['SNR'], index=y_df.columns)
#     snr['SNR'] = (snr['SNR'] - snr['SNR'].min()) / (snr['SNR'].max() - snr['SNR'].min()) #line to rescale SNR value from 0 to 1
    return snr

In [13]:
"""
Function that generates final pd.DataFrame with all features

Parameters: 

full_df: Dataframe
Dataframe with fourier spectrum

n: int
number of peaks

th1: float, from 0 to 1
threshhold

th2: float, from 0 to 1
threshhold

path: str
path to save the matrix

download: bool
download or not download feature matrix

Returns: 

features: DataFrame
features for full_df dataframe
"""

def create_feature_df(full_df, n, th1, th2, path, name, download=False):    
    properties = concat(full_df=full_df, n=n) 
    num_peak_1 = number_of_peaks(full_df=full_df, th=th1)
    num_peak_2 = number_of_peaks(full_df=full_df, th=th2)
    snr = SNR(df=full_df)
    target = pd.DataFrame(full_df[full_df.columns[::2]].loc['target'].reset_index().drop('index',axis=1))
    
    features = pd.concat([num_peak_1, num_peak_2, properties, snr, target], axis=1)
    
    if download == True: 
        features.to_csv(path + name + '.csv')
        return(features)
    else:
        return(features)

In [14]:
def electode_optical_matrix(electrode_df, optical_df, path, name, download=False):
    electrode_df.drop(['target'], axis=1, inplace=True)
    electrode_df.drop(['Label for OM and EL'], axis=1, inplace=True)
    el_om_features = pd.concat([electrode_df, optical_df], axis=1)
    el_om_features = el_om_features[el_om_features['Label for OM and EL'] == 1]
    el_om_features.drop(['Label for OM and EL'], axis=1, inplace=True)
    el_om_features = el_om_features.reset_index().drop('index',axis=1)

    if download == True: 
        el_om_features.to_csv(path + name + '.csv', index=False)
        return(el_om_features)
    else:
        return(el_om_features)

In [15]:
def electode_matrix(electrode_df, path, name, download=False):
    electrode_df = electrode_df[electrode_df['Label for OM and EL'] == 1]
    electrode_df.drop(['Label for OM and EL'], axis=1, inplace=True)
    electrode_df = electrode_df.reset_index().drop('index',axis=1)
    
    if download == True: 
        electrode_df.to_csv(path + name + '.csv', index=False)
        return(electrode_df)
    else:
        return(electrode_df)

In [18]:
def download_feature_matrices(path, th1, th2, dn_type):
    
    for i in tqdm_notebook(range(2, 6)):
        electrode_df = create_feature_df(mem_spec, n=i, th1=th1, th2=th2, path=path,\
                                         name='\Feature matrix EL ' + str(i) + ' peaks' + dn_type,  download=True)
#         electode_matrix(electrode_df, path=path, name='\Feature matrix EL short ' + str(i) + ' peaks', download=True)
        print(electrode_df.shape)
        optical_df = create_feature_df(niom_spec, n=i, th1=th1, th2=th2, path=path,\
                                       name='\Feature matrix OM ' + str(i) + ' peaks' + dn_type,  download=False)
        print(optical_df.shape)
        el_om_features = electode_optical_matrix(electrode_df, optical_df,\
                                                 path=path,\
                                                 name='\Feature matrix EL+OM ' + str(i) + ' peaks' + dn_type,\
                                                 download=True)
        print(el_om_features.shape)

In [19]:
path = r'C:\Users\ecath\Desktop\Research\Raw Data\LD dataset features'
if not os.path.exists(path):
    os.mkdir(path)
download_feature_matrices(path, 2, 3, '')

C:\Users\ecath\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\ecath\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\ecath\Anaconda3\lib\site-packages\pandas\core\indexes\range.py:465: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  return self._int64index.union(other)
C:\Users\ecath\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


(3456, 14)
(3456, 14)


C:\Users\ecath\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


(1728, 25)
(3456, 18)
(3456, 18)
(1728, 33)
(3456, 22)
(3456, 22)
(1728, 41)
(3456, 26)
(3456, 26)
(1728, 49)



-------------------------------------------------------